## 8.7 Doc clustering

In [4]:
# importing all data files and iterate
# to create 1)filename, 2)opinion_text
# In terms of opinion_text, you should make dataframe as string and append 

import pandas as pd
import glob, os

path = '/Users/yoontaepark/PerfectGuide/data/topics'

all_files = glob.glob(os.path.join(path, '*.data'))
# print(all_files)

filename_list, opinion_text = [], []

# pd.read_table: reading .data file
for file_ in all_files:
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]
    filename_list.append(filename)
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename': filename_list, 'opinion_text': opinion_text})
document_df.head(3)


,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,gas_mileage_toyota_camry_2007,...
2,room_holiday_inn_london,...


In [2]:
from nltk.stem import WordNetLemmatizer
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [46]:
# string.punctuation: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
# ord(!) == 33, so creating a dictionary of (33, None), (34, None), ... (126, None)

from nltk.stem import WordNetLemmatizer
import nltk
import string

# in terms of symbols, translate into None
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# def LemTokens(tokens):
#     return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

In [47]:
# using tfidf to fit & transform
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

In [72]:
document_df['opinion_text'].shape, feature_vect.shape

((51,), (51, 4466))

In [63]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [66]:
document_df['cluster_label'] = cluster_label
document_df.head(3)

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,2
1,gas_mileage_toyota_camry_2007,...,1
2,room_holiday_inn_london,...,0


In [68]:
# clustering issue: need to interpret by myself, this is about hotel
document_df[document_df['cluster_label'] == 0].sort_values(by='filename')

,filename,opinion_text,cluster_label
31,bathroom_bestwestern_hotel_sfo,...,0
17,food_holiday_inn_london,...,0
32,food_swissotel_chicago,...,0
49,free_bestwestern_hotel_sfo,...,0
39,location_bestwestern_hotel_sfo,...,0
3,location_holiday_inn_london,...,0
50,parking_bestwestern_hotel_sfo,...,0
2,room_holiday_inn_london,...,0
46,rooms_bestwestern_hotel_sfo,...,0
30,rooms_swissotel_chicago,...,0


In [69]:
# gps, windows, computer 
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
33,accuracy_garmin_nuvi_255W_gps,...,4
34,directions_garmin_nuvi_255W_gps,...,4
48,display_garmin_nuvi_255W_gps,...,4
21,features_windows7,...,4
10,satellite_garmin_nuvi_255W_gps,...,4
8,screen_garmin_nuvi_255W_gps,...,4
6,speed_garmin_nuvi_255W_gps,...,4
19,updates_garmin_nuvi_255W_gps,...,4
5,voice_garmin_nuvi_255W_gps,...,4


In [ ]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

In [74]:
# clustering by 3

from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
48,display_garmin_nuvi_255W_gps,...,0
44,fonts_amazon_kindle,...,0
41,price_amazon_kindle,...,0
40,speed_windows7,...,0
38,navigation_amazon_kindle,...,0
37,screen_netbook_1005ha,...,0
36,eyesight-issues_amazon_kindle,...,0
34,directions_garmin_nuvi_255W_gps,...,0
33,accuracy_garmin_nuvi_255W_gps,...,0


In [75]:
feature_vect.shape

(51, 4466)

In [76]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4466)
[[0.01015213 0.0012744  0.         ... 0.00735399 0.         0.        ]
 [0.         0.0009331  0.00337201 ... 0.         0.         0.        ]
 [0.         0.00433978 0.         ... 0.         0.00183257 0.00154897]]


In [97]:
km_cluster.cluster_centers_[:2].argsort()[:, ::-1]

array([[3520,  635,  641, ..., 2226, 2225, 2232],
       [2172, 2639, 3562, ..., 2703, 2702,    0]])

In [98]:
# integrated, so not using this
km_cluster.cluster_centers_.argsort()[::-1]

array([[   0, 1549, 1548, ..., 3438, 2020, 3596],
       [   0, 2702, 2703, ..., 3562, 2639, 2172],
       [2232, 2225, 2226, ...,  641,  635, 3520]])

In [128]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [187]:
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # [:, ::-1] is to sort by each list row 
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]
    
    for i in range(clusters_num):
        cluster_details[i] = {}
        cluster_details[i]['cluster'] = i
        
        top_feature_indexes = centroid_feature_ordered_ind[i, :top_n_features]
        
        # finding values based on index (to make this work, we need to use argsort to sort index)
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        top_features_values = cluster_model.cluster_centers_[i, top_feature_indexes].tolist()
        filenames = cluster_data[cluster_data['cluster_label'] == i]['filename']
        filenames = filenames.values.tolist()
        
        cluster_details[i]['top_features'] = top_features
        cluster_details[i]['top_features_value'] = top_features_values
        cluster_details[i]['filenames'] = filenames        
    
    return cluster_details    

In [188]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('### Cluster {}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews file name:', cluster_detail['filenames'][:6])
        print('==========================================')

In [189]:
# printing results
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df, \
                                     feature_names = feature_names, clusters_num =3, top_n_features = 10)

print_cluster_details(cluster_details)

### Cluster 0
Top features: ['screen', 'battery', 'battery life', 'keyboard', 'life', 'kindle', 'directions', 'size', 'voice', 'video']
Reviews file name: ['battery-life_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'size_asus_netbook_1005ha', 'screen_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle']
### Cluster 1
Top features: ['interior', 'mileage', 'seats', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews file name: ['gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'transmission_toyota_camry_2007', 'seats_honda_accord_2008', 'mileage_honda_accord_2008']
### Cluster 2
Top features: ['service', 'hotel', 'rooms', 'staff', 'room', 'food', 'location', 'clean', 'bathroom', 'parking']
Reviews file name: ['room_holiday_inn_london', 'location_holiday_inn_london', 'staff_bestwestern_hotel_sfo', 'service_swissotel_hotel_chicago', 'service_bestwestern_hotel_sfo', 'food_holida